<a href="https://colab.research.google.com/github/wind-yu/lhy-ML/blob/main/lhy_hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# From Google drive
# !gdown --id '1T0RPnu-Sg5eIPwQPfYysipfcz81MnsYe' --output Dataset.zip
# !unzip Dataset.zip

# From Dropbox
# If Dropbox is not work. Please use google drive.
!wget https://www.dropbox.com/s/vw324newiku0sz0/Dataset.tar.gz.aa?dl=0
!wget https://www.dropbox.com/s/z840g69e7lnkayo/Dataset.tar.gz.ab?dl=0
!wget https://www.dropbox.com/s/hl081e1ggonio81/Dataset.tar.gz.ac?dl=0
!wget https://www.dropbox.com/s/fh3zd8ow668c4th/Dataset.tar.gz.ad?dl=0
!wget https://www.dropbox.com/s/ydzygoy2pv6gw9d/Dataset.tar.gz.ae?dl=0
!cat Dataset.tar.gz.* | tar zxf -

--2023-05-24 03:27:37--  https://www.dropbox.com/s/vw324newiku0sz0/Dataset.tar.gz.aa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/vw324newiku0sz0/Dataset.tar.gz.aa [following]
--2023-05-24 03:27:37--  https://www.dropbox.com/s/raw/vw324newiku0sz0/Dataset.tar.gz.aa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd2dcffa62c09ae63516aad9164.dl.dropboxusercontent.com/cd/0/inline/B8rbu3Rs0nae6MK3acYX78zLQFl2YdyPf-ITaUHYeE7AO9ID6ORClC0C-Wvpdtt2d4R9DsoDKwT3W_vO0vhBOFUTqy-_eAN_-VT48XJvllaErevWo0yTm-FtMpenj3wvBIScsWSIxjnL7dQZh7E4VF5UTSQnv3Bl624EH2u5EnjVyQ/file# [following]
--2023-05-24 03:27:37--  https://ucd2dcffa62c09ae63516aad9164.dl.dropboxusercontent.com/cd/0/inline/B8rbu3Rs0nae6MK3acYX78zLQFl2YdyPf-ITaUHYeE7AO9ID6ORClC0C-Wvp

In [28]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker name to their corresponding id
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        # Get the total number of speakers
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances['feature_path'], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram
        mel = torch.load(os.path.join(self.data_dir, feat_path))
        
        # Segment mel-spectrogram into "segment_len" frames
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames
            mel = torch.FloatTensor(mel[start:start+self.segment_len])

        else:
            mel = torch.FloatTensor(mel)

        # Turn the speaker id into long for computing loss later
        speaker = torch.FloatTensor([speaker]).long()
        
        return mel, speaker

    def get_speaker_num(self):
        return self.speaker_num
        

In [29]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    # Process features within a batch
    """Collate a batch of data

    Args:
        batch (_type_): _description_
    """
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lenghts the same
    mel = pad_sequence(mel, batch_first=True, padding_value=-20) # pad log 10^(-20) which is very small value
    # mel: (batch_size, length, 40)
    return mel, torch.FloatTensor(speaker).long()

def get_dataloader(data_dir, batch_size, n_workers=0):
    """Generate dataloader

    Args:
        data_dir (_type_): _description_
        batch_size (_type_): _description_
        n_workers (int, optional): _description_. Defaults to 0.
    """
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_num()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size = batch_size,
        shuffle = True,
        drop_last = True,
        num_workers = n_workers,
        pin_memory = True,
        collate_fn = collate_batch
    )
    valid_loader = DataLoader(
        validset,
        batch_size = batch_size,
        num_workers = n_workers,
        drop_last = True,
        pin_memory = True,
        collate_fn = collate_batch
    )
    
    return train_loader, valid_loader, speaker_num


In [30]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model
        self.prenet = nn.Linear(40, d_model)

        # TODO:
        #   Change Transformer to Conformer
        #   https://arxiv.org/abs/2005.08100
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, dim_feedforward=256, nhead=2
        )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the dimension of features from d_model into speaker nums
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_spks)
        )
        
    def forward(self, mels):
        """forward

        Args:
            mels (_type_): (batch_size, length, 40)
        """
        # out: (batch_size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch_size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch_size, d_model)
        out = self.encoder_layer(out)
        # out: (batch_size, length, d_model)
        out = out.transpose(0, 1)
        # mean pooling
        stats = out.mean(dim=1)

        # out: (batch, n_spks)
        out = self.pred_layer(stats)

        return out

In [31]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1
):
    """Craete a schdule with a learning rate that decreases following the values of the cosine function between the initial lr set in 
    the optimizer to 0, after a warmup period during which it increases linearly between 0 and the initial lr set in the optimizer

    Args:
        optimizer (Optimizer): _description_
        num_warmup_step (int): _description_
        num_training_steps (int): _description_
        num_cycles (float, optional): _description_. Defaults to 0.5.
        last_epoch (int, optional): _description_. Defaults to -1.
    """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )
        
    return LambdaLR(optimizer, lr_lambda, last_epoch)

In [32]:
import torch

def model_fn(batch, model, criterion, device):
    """Forward a batch through the model

    Args:
        batch (_type_): _description_
        model (_type_): _description_
        criterion (_type_): _description_
        device (_type_): _description_
    """
    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)
    loss = criterion(outs, labels)

    # Get the speaker id with highest probability
    preds = outs.argmax(1)
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

In [34]:
from tqdm.auto import tqdm
import torch

def valid(dataloader, model, criterion, device):
    """Validate on validation set

    Args:
        dataloader (_type_): _description_
        model (_type_): _description_
        criterion (_type_): _description_
        device (_type_): _description_
    """
    model.eval()
    running_loss = 0.
    running_accuracy = 0.
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss = f"{running_loss / (i+1):.2f}",
            accuracy = f"{running_accuracy / (i+1):.2f}"
        )
    pbar.close()
    model.train()
    
    return running_accuracy / len(dataloader)

In [ ]:
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split

def parse_args():
    """arguments
    """
    config = {
        "data_dir": "./Dataset",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 1,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 70000,
    }
    
    return config

def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps
):
    """Main function

    Args:
        data_dir (_type_): _description_
        save_path (_type_): _description_
        batch_size (_type_): _description_
        n_workers (_type_): _description_
        valid_steps (_type_): _description_
        warmup_steps (_type_): _description_
        total_steps (_type_): _description_
        save_steps (_type_): _description_
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!", flush=True)

    model = Classifier(n_spks=speaker_num).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    print(f"[Info]: Finish creating model!", flush=True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=None, desc="Train", unit=" step")
    
    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Update model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss = f"{batch_loss:.2f}",
            accuracy = f"{batch_accuracy:.2f}",
            step = step + 1,
        )
        
        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()
            
            valid_accuracy = valid(valid_loader, model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()
                
            pbar = tqdm(total=valid_steps, ncols=None, desc="Train", unit=" step")

        # Save the best model so far
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()
    
if __name__ == '__main__':
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train:   0%|          | 0/2000 [00:00<?, ? step/s]